In [1]:
import pandas as pd
from string import Template
import random

In [2]:
#######################################################
# CONFIGURATION
#######################################################
n_local = 3
n_global = 3

In [3]:
g_expl = pd.read_csv('C:/datos/AIP/proyectos/XAIoGraphs/data/why/examples_titanic_binario/global_explainability.csv')
g_nodes = pd.read_csv('C:/datos/AIP/proyectos/XAIoGraphs/data/why/examples_titanic_binario/global_graph_nodes.csv')
l_expl = pd.read_csv('C:/datos/AIP/proyectos/XAIoGraphs/data/why/examples_titanic_binario/local_explainability.csv')
l_nodes = pd.read_csv('C:/datos/AIP/proyectos/XAIoGraphs/data/why/examples_titanic_binario/local_graph_nodes.csv')
why_ele = pd.read_csv('C:/datos/AIP/proyectos/XAIoGraphs/data/why/why_element.csv')
why_tar = pd.read_csv('C:/datos/AIP/proyectos/XAIoGraphs/data/why/why_target.csv')

In [4]:
print(g_expl.shape)
g_expl.head(5)

(8, 3)


,feature_name,feature_importance,feature_weight
0,title,0.043527,5
1,sex,0.043064,5
2,age,0.036054,4
3,fare,0.035937,4
4,pclass,0.025564,3


In [5]:
print(g_nodes.shape)
g_nodes.head(5)

(85, 5)


,target,node_name,node_importance,node_weight,rank
0,non-survivors,title_Mr,0.232662,35,1
1,non-survivors,sex_male,0.222662,35,2
2,non-survivors,pclass_3.00,0.202662,30,3
3,non-survivors,fare_0.05,0.162662,30,4
4,non-survivors,sex_female,0.142662,25,5


In [6]:
print(l_expl.shape)
l_expl.head(5)

(100, 10)


,id,age,fare,family_size,embarked,sex,pclass,title,is_alone,target
0,5,0.016229,0.146346,0.080302,0.058883,0.022773,0.194940,0.006434,0.098997,survivors
1,24,0.004413,0.160773,0.004618,-0.011278,0.166721,0.134596,0.164181,0.000880,survivors
2,50,0.009284,0.214948,0.042877,0.054373,0.115596,0.077795,0.114133,-0.004100,survivors
3,62,0.086006,-0.001297,-0.058085,0.098606,0.154625,-0.081196,0.162644,0.013792,non-survivors
4,78,0.020569,0.066509,0.056032,0.059268,0.165247,0.077380,0.164399,0.015500,survivors


In [7]:
print(l_nodes.shape)
l_nodes.head(5)

(800, 6)


,id,node_name,node_importance,node_weight,rank,target
0,5,pclass_1.00,0.194940,20,1,survivors
1,5,fare_0.75,0.146346,15,2,survivors
2,5,is_alone_1.00,0.098997,15,3,survivors
3,5,family_size_0.00,0.080302,15,4,survivors
4,5,embarked_S,0.058883,10,5,survivors


In [8]:
print(why_ele.shape)
why_ele.head(5)

(38, 2)


,feature,reason
0,sex_male,ser hombre
1,sex_female,ser mujer
2,pclass_1.00,viajar en 1ª clase
3,pclass_2.00,viajar en 2ª clase
4,pclass_3.00,viajar en 3ª clase


In [9]:
print(why_tar.shape)
why_tar.head(5)

(76, 3)


,target,feature,reason
0,non-survivors,sex_male,han muerto muchos hombres
1,non-survivors,sex_female,ser mujer
2,non-survivors,pclass_1.00,pocos viajar en 1ª clase
3,non-survivors,pclass_2.00,algunos viajar en 2ª clase
4,non-survivors,pclass_3.00,muchos viajar en 3ª clase


In [10]:
id = 62
df = (l_expl[l_expl['id'] == id]
      .merge(l_nodes[l_nodes['id'] == id].drop(columns=['target']).rename(columns={'rank': 'node_rank'}), on='id', how='inner')
      .merge(why_ele.rename(columns={'feature': 'node_name'}), on='node_name', how='inner')
      .merge(why_tar.rename(columns={'feature': 'node_name'}), on=['target', 'node_name'], how='inner', suffixes=['_local', '_global'])
      .sort_values(by='node_importance', ascending=False)
     )
df

,id,age,fare,family_size,embarked,sex,pclass,title,is_alone,target,node_name,node_importance,node_weight,node_rank,reason_local,reason_global
0,62,0.086006,-0.001297,-0.058085,0.098606,0.154625,-0.081196,0.162644,0.013792,non-survivors,sex_male,0.154625,20,2,ser hombre,han muerto muchos hombres
1,62,0.086006,-0.001297,-0.058085,0.098606,0.154625,-0.081196,0.162644,0.013792,non-survivors,embarked_S,0.098606,15,3,embarcar en un pueblo de clase baja,muchos embarcaron en Southampton
2,62,0.086006,-0.001297,-0.058085,0.098606,0.154625,-0.081196,0.162644,0.013792,non-survivors,age_0.95,0.086006,15,4,ser una persona muy mayor,eran muy mayores
3,62,0.086006,-0.001297,-0.058085,0.098606,0.154625,-0.081196,0.162644,0.013792,non-survivors,is_alone_1.00,0.013792,10,5,viajar solo,viajaban solos
4,62,0.086006,-0.001297,-0.058085,0.098606,0.154625,-0.081196,0.162644,0.013792,non-survivors,fare_0.95,-0.001297,10,6,pagar mucho por el billete,pagaron mucho por el billete
5,62,0.086006,-0.001297,-0.058085,0.098606,0.154625,-0.081196,0.162644,0.013792,non-survivors,family_size_1.00,-0.058085,10,7,ser una familia de pocos miembros,eran de una familia de pocos miembros
6,62,0.086006,-0.001297,-0.058085,0.098606,0.154625,-0.081196,0.162644,0.013792,non-survivors,pclass_1.00,-0.081196,15,8,viajar en 1ª clase,pocos viajar en 1ª clase


In [11]:
# Possible high-level templates (to choose only one)
templates_why = {
    0: "Por $temp_local_explain, este caso ha sido clasificado como $target, teniendo en cuenta que $temp_global_explain.",
    1: "Por $temp_local_explain, este caso ha sido clasificado como $target, puesto que $temp_global_explain.",
    2: "Este caso ha sido clasificado como $target por $temp_local_explain, ya que $temp_global_explain.",
    3: "La clasificación de este caso como $target se debe a $temp_local_explain, ya que $temp_global_explain.",
    4: "Como $temp_global_explain, y este caso se caracteriza por $temp_local_explain, ha sido clasificado como $target.",
}

In [12]:
def build_template(items, sep=", ", sep_last=" y "):
    i_list = ['$' + i for i in (items if isinstance(items, list) else list(items))]
    return (sep.join(i_list[:-1]) + sep_last + i_list[-1]) if len(i_list) > 1 else i_list[0]

In [17]:
kw_local = dict([('v_local_' + str(i), v) for i, v in enumerate(df['reason_local'].iloc[:n_local])])
kw_global = dict([('v_global_' + str(i), v) for i, v in enumerate(df['reason_global'].iloc[:n_global])])
temp_local_explain = build_template(kw_local)
temp_global_explain = build_template(kw_global)

random_template = random.randint(0, len(templates_why) - 1)
# random_template = 2
temp_why_str = (Template(Template(templates_why[random_template])
                         .substitute(temp_local_explain=temp_local_explain,
                                     temp_global_explain=temp_global_explain,
                                     target=df['target'].iloc[0]))
                .substitute(**kw_local, **kw_global)
                .capitalize())
print(temp_why_str)

Como han muerto muchos hombres, muchos embarcaron en southampton y eran muy mayores, y este caso se caracteriza por ser hombre, embarcar en un pueblo de clase baja y ser una persona muy mayor, ha sido clasificado como non-survivors.


In [ ]:
# TODO:
# - Default sentence if confidence level is below a given value
# - Language selection
# - Files' paths as constructor arguments
# Templates' file (same directory as code)